# Exemplo 20: Educação - Monitoramento de Exames (Dask)

Este notebook demonstra a análise de logs de exames online para detectar comportamentos suspeitos usando **Dask**.

**Cenário**: Identificar estudantes que mudam de aba (tab switch) ou respondem muito rápido (tempo < 5s) com frequência.

## 1. Configuração

In [ ]:
# Instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar e Instalar Spark
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz && tar xf spark-3.5.0-bin-hadoop3.tgz

# Baixar e Instalar Kafka
!wget https://archive.apache.org/dist/kafka/3.6.1/kafka_2.13-3.6.1.tgz && tar xf kafka_2.13-3.6.1.tgz

# Instalar pacotes Python
!pip install -q findspark pyspark kafka-python

In [ ]:
!pip install -q dask[complete] distributed

In [ ]:
from dask.distributed import Client
client = Client()
client

## 2. Gerar Logs de Exames

In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os

os.makedirs('exam_data', exist_ok=True)

def generate_exam_logs(exam_id):
    num_events = 5000
    students = [f'STUDENT-{i}' for i in range(1001, 1051)]
    
    df = pd.DataFrame({
        'exam_id': f'EXAM-{exam_id}',
        'student_id': np.random.choice(students, num_events),
        'event_type': np.random.choice(['ANSWER', 'TAB_SWITCH', 'MOUSE_OUT'], num_events, p=[0.8, 0.1, 0.1]),
        'time_spent_sec': np.random.exponential(60, num_events)
    })
    
    # Simular comportamento suspeito do STUDENT-1042
    mask = df['student_id'] == 'STUDENT-1042'
    df.loc[mask, 'event_type'] = np.random.choice(['ANSWER', 'TAB_SWITCH'], mask.sum(), p=[0.4, 0.6])
    df.loc[mask, 'time_spent_sec'] = np.random.exponential(5, mask.sum())
    
    df.to_csv(f'exam_data/exam_log_{exam_id}.csv', index=False)

print("Gerando logs de exames...")
for i in range(1, 11):
    generate_exam_logs(i)

## 3. Análise de Fraude

In [ ]:
ddf = dd.read_csv('exam_data/exam_log_*.csv')

# Contar eventos suspeitos por aluno
stats = ddf.groupby('student_id').agg(
    tab_switches=('event_type', lambda x: (x == 'TAB_SWITCH').sum()),
    fast_answers=('time_spent_sec', lambda x: (x < 10).sum())
)

computed_stats = stats.compute()

# Flag se mudar de aba > 20 vezes OU respostas rápidas > 30
flags = computed_stats[(computed_stats['tab_switches'] > 20) | (computed_stats['fast_answers'] > 30)]

print("Estudantes sob Revisão:")
print(flags)

In [ ]:
client.close()